In [1]:
%cd src

/mnt/e/Codes/Deep-SVDD/src


In [2]:
from datasets.main import load_dataset

In [ ]:
dataset_name = 'custom'
data_path = '../data/data_20.npy' 
normal_class= 3
dataset_custom = load_dataset(dataset_name, data_path, normal_class)

In [ ]:
dataset_name = 'mnist'
data_path = '../data' 
normal_class= 3
dataset_mnist = load_dataset(dataset_name, data_path, normal_class)

In [ ]:
dataset_mnist[0]

In [3]:
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
!pwd

In [4]:
total_data = np.load('/mnt/e/FLARACC/Sukanya_data/data_20.npy')

In [ ]:
total_data=total_data[:50]

In [ ]:
total_data.shape

In [15]:

class FaceLandmarksDataset(TorchvisionDataset):
    """Face Landmarks dataset."""

    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform

    def __len__(self):
        #return len(self.landmarks_frame)
        return total_data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

#         print(total_data.shape)
#         print(total_data[idx].shape)
        sample=total_data[idx]
        sample = np.array([sample])
        sample = sample.astype('float')
        if self.transform:
            sample = self.transform(sample)

        return sample

In [16]:
dataset = FaceLandmarksDataset(transform)

In [ ]:
dataloader = DataLoader(dataset, batch_size=10,
                        shuffle=True, num_workers=0)

In [6]:
import torch


In [ ]:
for x in dataloader:
    print(len(x))
    print(x.shape)
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(x[1])
    break


In [17]:
from datasets.preprocessing import get_target_label_idx, global_contrast_normalization

import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1'))
                               ])
train_set = FaceLandmarksDataset(transform=transform)

In [18]:
dataLoader = DataLoader(train_set, batch_size=10,
                        shuffle=True, num_workers=0)

In [19]:
iterCount = 0
datasetValues = {}

for data in dataLoader:
    if iterCount == 0:
        datasetValues['currDataset'] = data.clone()
    else:
        datasetValues['currDataset'] = torch.cat((datasetValues['currDataset'], data.clone()), 0)

    iterCount += 1


In [20]:
datasetValues['currDataset'].shape

torch.Size([20, 608, 1, 184])

In [21]:
datasetValues['currDataset'].min()

tensor(-4.2406, dtype=torch.float64)

In [22]:
datasetValues['currDataset'].max()

tensor(1.9455, dtype=torch.float64)

In [ ]:
len(dataLoader)

In [ ]:
for data in dataLoader:
    print(data)
    break

In [ ]:
test = torch.empty((608,1,184))

In [ ]:
test.shape

In [ ]:
test.reshape(1,184,608).shape

In [ ]:
train_loaders, _ = dataset_mnist.loaders(batch_size=10,
                        num_workers=0)

In [15]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from base.torchvision_dataset import TorchvisionDataset
from datasets.preprocessing import get_target_label_idx, global_contrast_normalization
import torch
import numpy as np



import torchvision.transforms as transforms


class Custom_Dataset(TorchvisionDataset):

    def __init__(self, root: str, normal_class=0):
        super().__init__(root)

        # Pre-computed min and max values (after applying GCN) from train data per class
        min_max = [-8.5016, 7.0120]

        # Custom dataset preprocessing: GCN (with L1 norm) and min-max feature scaling to [0,1]
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
                                        transforms.Normalize([min_max[0]],
                                                             [min_max[1] - min_max[0]])])

    
        self.train_set = Custom(root = self.root, transform=transform)
        self.test_set=None

    def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0):
        """Implement data loaders of type torch.utils.data.DataLoader for train_set and test_set."""

        print(f'train_set: {len(self.train_set)}')

        train_loader = DataLoader(self.train_set, batch_size=batch_size,
                        shuffle=shuffle_train, num_workers=num_workers)
        # test_dataLoader = DataLoader(self.test_set, batch_size=batch_size,
        #                 shuffle=shuffle_test, num_workers=num_workers)

        test_loader = train_loader
        print("***********in Loader***************")
        return train_loader, test_loader

class Custom(Dataset):
    """FLARACC dataset."""

    def __init__(self, root, transform=None):
        """
        Args:
            root (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = np.load(root)
        self.transform = transform
        print(f'data Shape: {self.data.shape}')

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        print(f'idx: {idx}')
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample=self.data[idx]
        sample = np.array([sample])
        sample = sample.astype('float')
        if self.transform:
            sample = self.transform(sample).reshape(1,184,608)

        print(f'sample shape: {sample.shape}')
        return sample, [0], [0]


In [16]:
dataset=Custom_Dataset(root='../data/data_20.npy')

data Shape: (20, 184, 608)


In [17]:
train_loader, _ = dataset.loaders(batch_size=10,
                        shuffle_train=True, num_workers=0)

train_set: 20
***********in Loader***************


In [18]:
for data in train_loader:
    print(data)

idx: 11
sample shape: torch.Size([1, 184, 608])
idx: 13
sample shape: torch.Size([1, 184, 608])
idx: 8
sample shape: torch.Size([1, 184, 608])
idx: 2
sample shape: torch.Size([1, 184, 608])
idx: 14
sample shape: torch.Size([1, 184, 608])
idx: 0
sample shape: torch.Size([1, 184, 608])
idx: 3
sample shape: torch.Size([1, 184, 608])
idx: 7
sample shape: torch.Size([1, 184, 608])
idx: 19
sample shape: torch.Size([1, 184, 608])
idx: 1
sample shape: torch.Size([1, 184, 608])
[tensor([[[[ 0.4735,  0.4746,  0.4757,  ...,  0.5420,  0.5420,  0.5593],
          [ 0.5593,  0.5766,  0.5766,  ...,  0.7152,  0.7152,  0.7152],
          [ 0.7326,  0.7326,  0.7326,  ..., -0.0645, -0.0992, -0.1338],
          ...,
          [-1.2428, -1.2255, -1.1735,  ..., -0.1512, -0.1512, -0.1512],
          [-0.1512, -0.1685, -0.1685,  ..., -0.4284, -0.4284, -0.4284],
          [-0.4457, -0.4457, -0.4631,  ..., -2.2478, -2.2652, -2.2652]]],


        [[[ 0.4735,  0.4746,  0.4757,  ...,  0.5420,  0.5420,  0.5593],
  